In [1]:
import gradio as gr
import clickhouse_connect
import requests
import json
import gc

import clickhouse_connect
import requests
import json
import requests
import uuid
from environs import Env 
from openai import OpenAI
from embedding import E5LargeEmbeddingFunction
from similarity import prep_query,_clickhouse_query_window
from langchain_community.chat_models import GigaChat
from langchain_core.messages import HumanMessage, SystemMessage

/home/veidlink/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
env = Env()                # Создаем экземпляр класса Env
env.read_env(path='.env')  # Методом read_env() читаем файл .env и загружаем из него переменные в окружение 

SBER_AUTH = env('SBER_AUTH')
OPENAI_API_KEY = env('OPENAI_API_KEY')

client = clickhouse_connect.get_client(host='fre1otkh1b.europe-west4.gcp.clickhouse.cloud', port=8443, username='default', password='.SAxpSMBh27IT')
emb_func = E5LargeEmbeddingFunction()

system_prompt = """
### SYSTEM ###
Ты - дружелюбный правовой ассистент.
Ты должен давать информативные ответы по сути вопроса только на основании предоставленного источика.
Если вопрос не относится к теме права, ты должен отвечать "Давай сменим тему."
Указывай в ответе статьи на которые ссылаешься, если приведены их номера.
"""

rag_prompt = """
### CONTEXT ###

##### ИСТОЧНИК #####
{page_text}

### USER ###
{QUESTION}

### ASSISTANT ###

"""

def message_llm(system_prompt, user_input): # Функция для общения с Мистраль
    '''Function to message (test) our LLM'''

    try:
        url = 'https://golden-oyster-possible.ngrok-free.app/v1/chat/completions'  
        

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            { "role": "user", "content": user_input}
          ],
          "temperature": 0.6,
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        print(f"Error {e}!")
        
def format_output(answer, page_num):        # Функция для форматирования ответов моделей
    output = f"{answer}"
    if not 'сменим тему' in answer:
        output += f"\n\nИспользованные страницы КоАП РФ: {', '.join(map(str, set(page_num)))}"
    return output

def get_gigatoken(auth_token, scope='GIGACHAT_API_PERS'):   # Функция для получения токена GigaChat
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка при получении токена для GigaChat: {str(e)}")
        return -1

response = get_gigatoken(SBER_AUTH)                         # Получение токена GigaChat
if response != 1:
  giga_token = response.json()['access_token']

# Инициализация моделей
gigachat = GigaChat(access_token=giga_token, verify_ssl_certs=False)    
chat_gpt = OpenAI(api_key=OPENAI_API_KEY)
model = 'gpt-3.5-turbo'

def ask_chatbot(mode, question):                            # Логика взаимодействия с 3 моделями, обернутая в функцию
    gc.collect()
    try:
        formatted_rag_prompt = ""
        query = prep_query(question)
        result = _clickhouse_query_window(query=query, client=client, emb_func=emb_func, limit_knn=1)

        page_text = ''
        page_num = []

        for source in result:    
            page_num.append(source[0])
            page_text += source[1]
        
        formatted_rag_prompt = rag_prompt.format(
            page_text=page_text,
            QUESTION=question
        )

        if mode == "Mistral 7b":
            mis_answer = message_llm(system_prompt, formatted_rag_prompt)
            return format_output(mis_answer, page_num)
            
        elif mode == "ChatGPT":
            try:
                completion = chat_gpt.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": formatted_rag_prompt}
                    ],
                    temperature=0.6,   # [0, 2] Higher values make output more random, lower values make it deterministic.
                    )
            except Exception as e:
                print(e)

            return format_output(completion.choices[0].message.content, page_num)
        

        elif mode == "GigaChat":
            messages = [
                SystemMessage(
                    content=system_prompt
                ),
                HumanMessage(
                    content=formatted_rag_prompt)
                ]

            giga_answer = gigachat.invoke(input=messages)

            return format_output(giga_answer.content, page_num)
    except Exception as e:
        return f"Ошибка. Попробуйте еще раз."  


iface = gr.Interface(                                       # GUI Gradio
    fn=ask_chatbot,
    inputs=[
        gr.Dropdown(choices=["Mistral 7b", "ChatGPT", "GigaChat"], label="Выберите режим"),
        gr.Textbox(lines=2, placeholder="Введите ваш вопрос здесь...", label="Вопрос")
    ],
    outputs=gr.Textbox(label="Ответ"),
    title="LLM-based RAG Fine Assistant",
    description="Выберите режим и введите ваше сообщение, чтобы получить ответ от модели:"
)

# Запуск интерфейса
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://3c761df8a0bd4f415a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
